In [ ]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import random
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [ ]:
from openpyxl.utils import escape
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
import os

In [ ]:
# Cài Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y ./google-chrome-stable_current_amd64.deb

--2025-05-05 01:47:09--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.202.91, 173.194.202.136, 173.194.202.93, ...
Connecting to dl.google.com (dl.google.com)|173.194.202.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116169896 (111M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 110.79M   266MB/s    in 0.4s    

2025-05-05 01:47:09 (266 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [116169896/116169896]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded,

In [ ]:
!google-chrome --version

Google Chrome 136.0.7103.59 


In [ ]:
!wget https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip
!unzip chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

--2025-05-05 01:47:50--  https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 192.178.163.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9593043 (9.1M) [application/zip]
Saving to: ‘chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>]   9.15M  --.-KB/s    in 0.1s    

2025-05-05 01:47:51 (84.2 MB/s) - ‘chromedriver-linux64.zip’ saved [9593043/9593043]

Archive:  chromedriver-linux64.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  


In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

In [ ]:
service = Service("/usr/bin/chromedriver")
browser = webdriver.Chrome(service=service, options=options)

In [ ]:
# Kiểm tra
browser.get("https://cafef.vn/doanh-nghiep.chn")
print(browser.title)

Doanh Nghiệp | Thông tin tài chính và hoạt động kinh doanh | CafeF


In [ ]:
def clean_text(text):
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub("", text)
    return text

In [ ]:
def get_content(original_file, destination_file):
  df = pd.read_excel(original_file)
  df['Content'] = ""
  for i in range(len(df)):
    text = ""
    url = df.url[i]
    try:
        #Mở và thu thập dữ liệu nội dung bài báo thông qua URL của bài báo
        browser.get(url)
        print("Đang xử lý URL: " + str(url))
        print("Đang xử lý bài báo thứ: " + str(i))
        try:
            title_news = browser.find_element("xpath","//div[@class='left_cate totalcontentdetail']/h1[@class='title']")
        except:
            title_news = None
            # browser.quit()
        if title_news is not None:
          try:
            # description = browser.find_element("xpath", "/html/body/form/div[2]/div[4]/div[5]/div/div[1]/div[" + str(j) + "]/h2")
            description = browser.find_element("xpath","//div[@class='w640 fr clear']/h2[@class='sapo'][1]")

          except:
              continue

          text = text + description.text + " "

          for k in range(1,30):
            try:
              article = browser.find_element("xpath", "//div[@class='detail-content afcbc-body']/p["+str(k)+"]")
              text = text + article.text
            except:
              break
          # df['Content'][i] = df['Content'][i] + text
          cleaned_text = clean_text(text)
          df.loc[i, 'Content'] = df.loc[i, 'Content'] + cleaned_text
    except Exception as e:
      print(f"Lỗi khi xử lý URL {url}: {e}")
      continue
  # Tạo thư mục nếu chưa tồn tại
  # os.makedirs(destination_file, exist_ok=True)
  with pd.ExcelWriter(destination_file, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
  print("Đã lưu file excel " + str(destination_file))

In [ ]:
get_content(original_file='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/SA/v1/data/url_title_new_vn30/train_data/BCM.xlsx', destination_file='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/SA/v1/data/content_news_vn30/train_data/BCM.xlsx')

Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1923206/becamex-bcm-hut-them-hon-400-ty-dong-trai-phieu-lai-suat-125nam.chn
Đang xử lý bài báo thứ: 0
Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1923928/bcm-ong-nguyen-van-hung-chu-tich-hdqt-da-mua-70000-cp.chn
Đang xử lý bài báo thứ: 1
Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1917556/ba-ong-lon-becamex-vsip-vrg-nghien-cuu-phat-trien-2-du-an-khung-tong-dien-tich-hon-2800ha-tai-khanh-hoa.chn
Đang xử lý bài báo thứ: 2
Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1918166/bcm-cbtt-ket-qua-chao-ban-trai-phieu.chn
Đang xử lý bài báo thứ: 3
Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1918167/bcm-cbtt-ve-viec-thay-doi-dia-chi-tru-so-chi-nhanh.chn
Đang xử lý bài báo thứ: 4
Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1914274/bcm-nghi-quyet-hdqt-thong-qua-phat-hanh-rieng-le-1300-ty-dong-trai-phieu-dn.chn
Đang xử lý bài báo thứ: 5
Đang xử lý URL: https://cafef.vn/du-lieu/bcm-1913108/bcm-nghi-quyet-hdqt-ve-viec-de-nghi-cap-tin-dung-tai-msb.chn
Đa

In [ ]:
def get_content_new_for_vn30(tickers):
  for ticker in tickers:
    print("Đang xử lý: " + str(ticker))
    original_file = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/SA/v1/data/url_title_new_vn30/train_data/'+ticker+'.xlsx'
    destination_file = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/SA/v1/data/content_news_vn30/train_data/'+ticker+'.xlsx'
    get_content(original_file, destination_file)
    print("Đã xử lý xong: " + str(ticker))

In [ ]:
# ,'SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
# tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
tickers = ['LPB','MBB','MSN','MWG','PLX']
get_content_new_for_vn30(tickers)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Đang xử lý bài báo thứ: 670
Đang xử lý URL: https://cafef.vn/du-lieu/mbb-1708981/cmbb1902-quyet-dinh-vv-thay-doi-dang-ky-niem-yet-chung-quyen-co-bao-dam.chn
Đang xử lý bài báo thứ: 671
Đang xử lý URL: https://cafef.vn/du-lieu/mbb-1649788/mbb-ket-qua-phat-hanh-trai-phieu-theo-phuong-thuc-rieng-le-04112019.chn
Đang xử lý bài báo thứ: 672
Đang xử lý URL: https://cafef.vn/du-lieu/mbb-1657906/mbb-cong-doan-co-so-mbb-dang-ky-nhan-chuyen-nhuong-27500-cp.chn
Đang xử lý bài báo thứ: 673
Đang xử lý URL: https://cafef.vn/du-lieu/mbb-1648549/mbb-thay-doi-logo-va-he-thong-nhan-dien-thuong-hieu-moi.chn
Đang xử lý bài báo thứ: 674
Đang xử lý URL: https://cafef.vn/du-lieu/mbb-1706434/mbb-chong-ba-nguyen-thi-thuy-tvhdqt-dang-ky-ban-292680-cp.chn
Đang xử lý bài báo thứ: 675
Đang xử lý URL: https://cafef.vn/du-lieu/mbb-1671028/cmbb1906-quyet-dinh-chap-thuan-dang-ky-niem-yet-chung-quyen-co-bao-dam-chung-quyen-cmbb1906.chn
Đang xử lý bài báo thứ: 676